In [ ]:
!pip install transformers

In [ ]:
!pip install adabelief-pytorch==0.2.0

In [ ]:
!pip install -U nbformat

In [ ]:
import os
import re
import numpy as np
from tqdm import tqdm
from glob import glob
import json
import requests
import tensorflow as tf
from transformers import BertModel, TFBertModel, TFRobertaModel, RobertaTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from adabelief_pytorch import AdaBelief
from transformers.optimization import get_cosine_schedule_with_warmup
from tqdm import tqdm, tqdm_notebook
import shutil
import gc
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import urllib.request
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [ ]:
dataset_url = "https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv"
dataset_name = "finance_data.csv"

In [ ]:
urllib.request.urlretrieve(dataset_url,
                           filename=dataset_name
                           )

('finance_data.csv', <http.client.HTTPMessage at 0x797921b64ca0>)

In [ ]:
dataset = pd.read_csv(dataset_name)
dataset.head()

,labels,sentence,kor_sentence
0,neutral,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [ ]:
dataset['labels'] = dataset['labels'].replace(['neutral','positive','negative'],[0,1,2])

dataset_prep_file = './sample_data/dataset_prep.csv'
dataset.drop_duplicates(subset=['kor_sentence'],inplace=True)
dataset.to_csv(dataset_prep_file)

In [ ]:
dataset = pd.read_csv('./sample_data/dataset_prep.csv')
dataset[:10]

,Unnamed: 0,labels,kor_sentence
0,0,0,"Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,1,0,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,2,2,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,3,1,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,4,1,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."
5,5,1,ASPOCOMP의 성장기에 대한 자금 조달은 기술적으로 더 까다로운 HDI 인쇄 회...
6,6,1,2010년 4분기 Componenta의 순매출은 전년 동기의 7600만 유로에서 2...
7,7,1,"2010년 3분기 순매출은 5.2%, 영업이익은 34.9% 증가한 23.5MN을 기..."
8,8,1,영업이익은 2007년 해당 기간의 8.7 mn에서 13.1 mn으로 증가하여 순매출...
9,9,1,영업이익은 총 21.1 유로로 2007년 18.6 mn에서 증가하여 순매출의 9.7...


In [ ]:
dataset.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
X_data = dataset['kor_sentence']
y_data = dataset['labels']

In [ ]:
TEST_SIZE = 0.2
RANDOM_STATE = 42
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data,
                                                    test_size = TEST_SIZE,
                                                    random_state = RANDOM_STATE,
                                                    stratify=y_data)

In [ ]:
print(f"# train-set : {len(X_train)}")
print(f"# test-set : {len(X_test)}")

# train-set : 3861
# test-set : 966


In [ ]:
y_train.value_counts(normalize=True)

,proportion
labels,
0,0.592852
1,0.282051
2,0.125097


In [ ]:
y_test.value_counts(normalize=True)

,proportion
labels,
0,0.592133
1,0.282609
2,0.125259


In [ ]:
X = pd.concat([X_train,y_train],axis=1)
X.reset_index(drop=True,inplace=True)
X_test = X_test.reset_index()
from transformers import AutoTokenizer

# Tokenizer 정의 및 초기화
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--klue--roberta-base/snapshots/02f94ba5e3fcb7e2a58a390b8639b0fac974a8da/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--klue--roberta-base/snapshots/02f94ba5e3fcb7e2a58a390b8639b0fac974a8da/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--klue--roberta-base/snapshots/02f94ba5e3fcb7e2a58a390b8639b0fac974a8da/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--klue--roberta-base/snapshots/02f94ba5e3fcb7e2a58a390b8639b0fac974a8da/tokenizer_config.json


In [ ]:
# TrainDataset 클래스
class TrainDataset(Dataset):
    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):
        sentence = self.df_data.loc[index, 'kor_sentence']
        encoded_dict = tokenizer(
            text=sentence,
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            truncation=True,
            return_tensors="pt")

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        target = torch.tensor(self.df_data.loc[index, "labels"])
        sample = (padded_token_list, token_type_id, att_mask, target)
        return sample

    def __len__(self):
        return len(self.df_data)

In [ ]:
# TestDataset 클래스
class TestDataset(Dataset):
    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):
        sentence = self.df_data.loc[index, 'kor_sentence']
        encoded_dict = tokenizer(
            text=sentence,
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            truncation=True,
            return_tensors="pt")

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        sample = (padded_token_list, token_type_id, att_mask)
        return sample

    def __len__(self):
        return len(self.df_data)

In [ ]:
train_data = TrainDataset(X)

test_data = TestDataset(X_test)

train_dataloader = torch.utils.data.DataLoader(train_data,
                                               batch_size = 32,
                                               shuffle=True,
                                               num_workers=0)

test_dataloader = torch.utils.data.DataLoader(test_data,
                                               batch_size = 32,
                                               shuffle=False,
                                               num_workers=0)

In [ ]:
# device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_EPOCHS = 5
max_grad_norm = 1.0
log_interval = 10
MAX_LEN = 512  # 최대 입력 시퀀스 길이 설정

# 모델 및 학습 설정
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-base", num_labels=3)

# 미세 조정
n = 0
for name, child in model.named_children():
    if n == 0:
        h = 0
        for param in child.parameters():
            if h <= 0:
                param.requires_grad = False
            h += 1
    n += 1

model.to(device)

# 옵티마이저 및 스케줄러 설정
optimizer = AdaBelief(model.parameters(), lr=1e-5, eps=1e-16, betas=(0.9, 0.999), weight_decouple=True, rectify=False)
warmup_ratio = 0.1
t_total = len(train_dataloader) * NUM_EPOCHS
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 학습 루프
for e in range(NUM_EPOCHS):
    train_acc = 0.0
    model.train()
    torch.set_grad_enabled(True)
    for batch_id, (input_id, token_type_id, attention_mask, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        input_id = input_id.to(device)
        token_type_id = token_type_id.to(device)
        attention_mask = attention_mask.to(device)
        label = label.to(device)

        outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask, labels=label)
        loss = outputs[0]
        out = outputs[1]

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()

        _, preds = torch.max(out, dim=1)
        train_acc += f1_score(preds.cpu(), label.cpu(), average='weighted')

        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))

    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--klue--roberta-base/snapshots/02f94ba5e3fcb7e2a58a390b8639b0fac974a8da/config.json
Model config RobertaConfig {
  "_name_or_path": "klue/roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.4

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief


  1%|          | 1/121 [00:04<09:36,  4.81s/it]

epoch 1 batch id 1 loss 1.0818251371383667 train acc 0.6382978723404256


  9%|▉         | 11/121 [00:35<05:33,  3.03s/it]

...

epoch 5 batch id 101 loss 0.15919943153858185 train acc 0.9512932722374032


 92%|█████████▏| 111/121 [05:22<00:29,  2.92s/it]

epoch 5 batch id 111 loss 0.18284404277801514 train acc 0.9523095542832649


100%|██████████| 121/121 [05:51<00:00,  2.90s/it]

epoch 5 batch id 121 loss 0.1450500786304474 train acc 0.9502044779048009
epoch 5 train acc 0.9502044779048009


In [ ]:
preds = []
model.eval()
torch.set_grad_enabled(False)
for batch_id, (input_id, token_type_id, attention_mask) in enumerate(tqdm(test_dataloader)):
        input_id = input_id.long().to(device)
        token_type_id = token_type_id.long().to(device)
        attention_mask = attention_mask.long().to(device)
        outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask)
        out = outputs[0]
        for inp in out:
          preds.append(inp.detach().cpu().numpy())
preds = np.array(preds)

100%|██████████| 31/31 [00:27<00:00,  1.13it/s]


In [ ]:
predicted_label = np.argmax(preds, axis=1)

In [ ]:
path2data='./metric'
if not os.path.exists(path2data):
  os.mkdir(path2data)
CL_REPORT_FILE = "./metric/cl_report.csv"

cl_report = classification_report(y_test, predicted_label, output_dict = True)
cl_report_df = pd.DataFrame(cl_report).transpose()
cl_report_df = cl_report_df.round(3)
cl_report_df.to_csv(CL_REPORT_FILE)
print(cl_report_df)

              precision  recall  f1-score  support
0                 0.892   0.883     0.888  572.000
1                 0.782   0.828     0.804  273.000
2                 0.883   0.810     0.845  121.000
accuracy          0.858   0.858     0.858    0.858
macro avg         0.852   0.840     0.846  966.000
weighted avg      0.860   0.858     0.859  966.000


In [ ]:
import os
import torch

# 저장할 경로의 디렉토리가 없으면 생성
save_path = './model/'
if not os.path.exists(save_path):
    os.makedirs(save_path)

# 모델 저장
torch.save(model.state_dict(), os.path.join(save_path, 'klue_base_fold3_s.pth'))
